In [25]:
import torch
from torch import nn
import gym

import numpy as np

In [7]:
env_id = 'CartPole-v1'
env = gym.make(env_id)
n_actions = env.action_space.n
len_obs_space = env.observation_space.shape[0]

print('n_actions = {}'.format(n_actions))
print('len_obs_space = {}'.format(len_obs_space))

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_actions = 2
len_obs_space = 4


https://gsurma.medium.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288#:~:text=Cartpole%20%2D%20known%20also%20as%20an,forces%20to%20a%20pivot%20point.


https://github.com/gsurma/cartpole

In [28]:
class DQN(nn.Module):

    def __init__(self, lr) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.seq_relu = nn.Sequential(
            nn.Linear(len_obs_space, len_obs_space),
            nn.ReLU(),
            nn.Linear(len_obs_space, len_obs_space),
            nn.ReLU(),
            nn.Linear(len_obs_space, n_actions)
        )
        self.learning_rate = lr
        self.optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        self.loss = nn.MSELoss()

    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.seq_relu(x)
        return logits

learning_rate = 0.05
q_approx = DQN(lr=learning_rate).to(device)
q_target = DQN(lr=learning_rate).to(device)
# load same weights as approx
q_target.load_state_dict(q_approx.state_dict())
q_target.eval()
print(q_approx)
print(q_target)


DQN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (seq_relu): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=2, bias=True)
  )
  (loss): MSELoss()
)
DQN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (seq_relu): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=2, bias=True)
  )
  (loss): MSELoss()
)


In [24]:
from collections import deque

MEMORY_SIZE = 100
D = deque(maxlen=MEMORY_SIZE)
for i in range(MEMORY_SIZE):
    D.append(0)


MINI_BATCH_SIZE = 5
EPISODES = 200

EPSILON_MIN = 0.2
EPSILON_START = 0.9
EPISLON_BASE = 0.995
epsilon = EPSILON_START

In [12]:
import random

class Agent:
    
    def __init__(self, q_model, q_target, env, n_actions, epsilon) -> None:
        self.q_model = q_model
        self.q_target = q_target
        self.env = env
        self.n_actions = n_actions
        self.epsilon = epsilon
        self.device = self.q_model.device

    def train_episode(self):
        done = False
        obs = self.env.reset()
        obs = torch.tensor(obs, device=self.device)

        while not done:
            # get action to execute based on state
            action = self.get_action(obs)
            #  take action, go to next time step
            obs_next, reward, done, info = self.env.step(action.item())

            # convert to tensors
            obs_next = torch.tensor(obs_next, device=self.device)
            reward = torch.tensor(reward, device=self.device)
            done = torch.tensor(done, device=self.device, dtype=torch.bool)
            
            # create transitions
            transition = (obs, action, reward, obs_next, done)
            # store transitions
            D.append(transition)

            # sample minibatch
            minibatch = random.choices(D, k=MINI_BATCH_SIZE)
            y_out = torch.zeros()
            for sample in minibatch:
                done_index = -1
                if sample[done_index]:
                    pass

            # Compute prediction and loss
            pred = model(X)
            loss = self.q_model.loss(pred, y)
            
            # Backpropagation
            self.q_model.optimizer.zero_grad()
            loss.backward()
            self.q_model.optimizer.step()
                
    def get_action(self, obs):
        sample = random.random()
        action = None
        if sample < self.epsilon:
            action = torch.tensor([random.choice(range(self.n_actions))])
        else:
            with torch.no_grad():
                q_vals = self.q_model(obs)
                print(q_vals)
                action = torch.argmax(q_vals)
        return torch.tensor([action], device=self.device)


In [6]:
for episode in range(EPISODES):
    # reset environment for new episode
    obs = env.reset()
    # get action to execute based on state
    # action = q_approx.get_action(obs)
    # update epsilon value